## 1、空值 特征处理

空值个数分桶时，对于训练集、测试集（其他数据集）都保证同一个分桶逻辑

In [ ]:
import pandas as pd


train_x = pd.read_csv("../data/train_x.csv")
test_x = pd.read_csv("../data/test_x.csv")
train_unlabeled =  pd.read_csv("../data/train_unlabeled.csv")

# 1、每一个样本（每行）小于0的个数
train_x['n_null'] = (train_x<0).sum(axis=1)
test_x['n_null'] = (test_x<0).sum(axis=1)
train_unlabeled['n_null'] = (train_unlabeled<0).sum(axis=1)

# 2、新特征 -- 空值个数分桶
train_x['discret_null'] = train_x.n_null
train_x.discret_null[train_x.discret_null<=32] = 1
train_x.discret_null[(train_x.discret_null>32)&(train_x.discret_null<=69)] = 2
train_x.discret_null[(train_x.discret_null>69)&(train_x.discret_null<=147)] = 3
train_x.discret_null[(train_x.discret_null>147)&(train_x.discret_null<=194)] = 4
train_x.discret_null[(train_x.discret_null>194)] = 5
train_x[['uid','n_null','discret_null']].to_csv('../data/train_x_null.csv',index=None)

test_x['discret_null'] = test_x.n_null
test_x.discret_null[test_x.discret_null<=32] = 1
test_x.discret_null[(test_x.discret_null>32)&(test_x.discret_null<=69)] = 2
test_x.discret_null[(test_x.discret_null>69)&(test_x.discret_null<=147)] = 3
test_x.discret_null[(test_x.discret_null>147)&(test_x.discret_null<=194)] = 4
test_x.discret_null[(test_x.discret_null>194)] = 5
test_x[['uid','n_null','discret_null']].to_csv('../data/test_x_null.csv',index=None)

train_unlabeled['discret_null'] = train_unlabeled.n_null
train_unlabeled.discret_null[train_unlabeled.discret_null<=32] = 1
train_unlabeled.discret_null[(train_unlabeled.discret_null>32)&(train_unlabeled.discret_null<=69)] = 2
train_unlabeled.discret_null[(train_unlabeled.discret_null>69)&(train_unlabeled.discret_null<=147)] = 3
train_unlabeled.discret_null[(train_unlabeled.discret_null>147)&(train_unlabeled.discret_null<=194)] = 4
train_unlabeled.discret_null[(train_unlabeled.discret_null>194)] = 5
train_unlabeled[['uid','n_null','discret_null']].to_csv('../data/train_unlabeled_null.csv',index=None)

## 2、连续特征 -- 排序特征

数据分别排序，使用的时候需要归一化

In [ ]:
import pandas as pd

feature_type = pd.read_csv('../data/features_type.csv')
numeric_feature = list(feature_type[feature_type.type=='numeric'].feature)

# rank特征的命名：在原始特征前加'r',如'x1'的rank特征为'rx1'

# 三份数据集分别排序，使用的时候需要归一化。
# 更合理的做法是merge到一起排序，这个我们也试过，效果差不多，因为数据分布相对比较一致。


test = pd.read_csv('../data/test_x.csv')[['uid']+numeric_feature]
test_rank = pd.DataFrame(test.uid,columns=['uid'])
for feature in numeric_feature:
    test_rank['r'+feature] = test[feature].rank(method='max')
test_rank.to_csv('../data/test_x_rank.csv',index=None)


train = pd.read_csv('../data/train_x.csv')[['uid']+numeric_feature]
train_rank = pd.DataFrame(train.uid,columns=['uid'])
for feature in numeric_feature:
    train_rank['r'+feature] = train[feature].rank(method='max')
train_rank.to_csv('../data/train_x_rank.csv',index=None)

## 3、连续特征 -- 根据排序分桶,离散化

In [ ]:
import pandas as pd

train = pd.read_csv("../data/train_x_rank.csv")
train_x = train.drop(['uid'],axis=1)
test = pd.read_csv("../data/test_x_rank.csv")
test_x = test.drop(['uid'],axis=1)
train_unlabeled =  pd.read_csv("../data/train_unlabeled_rank.csv")
train_unlabeled_x =  train_unlabeled.drop(['uid'],axis=1)


train_x[train_x<1500] = 1
train_x[(train_x>=1500)&(train_x<3000)] = 2
train_x[(train_x>=3000)&(train_x<4500)] = 3
train_x[(train_x>=4500)&(train_x<6000)] = 4
train_x[(train_x>=6000)&(train_x<7500)] = 5
train_x[(train_x>=7500)&(train_x<9000)] = 6
train_x[(train_x>=9000)&(train_x<10500)] = 7
train_x[(train_x>=10500)&(train_x<12000)] = 8
train_x[(train_x>=12000)&(train_x<13500)] = 9
train_x[train_x>=13500] = 10
#离散特征的命名：在原始特征前加'd',如'x1'的离散特征为'dx1'
rename_dict = {s:'d'+s[1:] for s in train_x.columns.tolist()}
train_x = train_x.rename(columns=rename_dict)
train_x['uid'] = train.uid
train_x.to_csv('../data/train_x_discretization.csv',index=None)

## 4、连续特征 -- 排序特征，分桶count

每一行数据，特征排序count

In [ ]:
import pandas as pd

train_x = pd.read_csv('../data/train_x_discretization.csv')
test_x = pd.read_csv('../data/test_x_discretization.csv')
train_unlabeled_x =  pd.read_csv('../data/train_unlabeled_discretization.csv')

train_x['n1'] = (train_x==1).sum(axis=1) # 一行数据中，特征排序为 1 的特征总数
train_x['n2'] = (train_x==2).sum(axis=1)
train_x['n3'] = (train_x==3).sum(axis=1)
train_x['n4'] = (train_x==4).sum(axis=1)
train_x['n5'] = (train_x==5).sum(axis=1)
train_x['n6'] = (train_x==6).sum(axis=1)
train_x['n7'] = (train_x==7).sum(axis=1)
train_x['n8'] = (train_x==8).sum(axis=1)
train_x['n9'] = (train_x==9).sum(axis=1)
train_x['n10'] = (train_x==10).sum(axis=1)
train_x[['uid','n1','n2','n3','n4','n5','n6','n7','n8','n9','n10']].to_csv('../data/train_x_nd.csv',index=None)